In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.sparse
import sklearn
from itertools import product
import warnings
warnings.filterwarnings('ignore')

In [2]:
sales            = pd.read_csv("../sales_train_v2.csv")
items            = pd.read_csv("../items.csv")
item_categories  = pd.read_csv("../item_categories.csv")
shops            = pd.read_csv("../shops.csv")
test             = pd.read_csv("../test.csv")

In [3]:
sales[sales.item_price>100000]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
1163158,13.12.2013,11,12,6066,307980.0,1.0


In [4]:
sales[sales.item_cnt_day>1000]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2909818,28.10.2015,33,12,11373,0.908714,2169.0


In [5]:
sales[(sales.shop_id ==0) & (sales.item_id == 30) & (sales.date_block_num == 1)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
173344,21.02.2013,1,0,30,265.0,2.0
173345,20.02.2013,1,0,30,265.0,2.0
173346,18.02.2013,1,0,30,265.0,4.0
173347,17.02.2013,1,0,30,265.0,4.0
173348,16.02.2013,1,0,30,265.0,9.0
173349,15.02.2013,1,0,30,265.0,2.0
173361,22.02.2013,1,0,30,265.0,2.0
173402,23.02.2013,1,0,30,265.0,3.0
173441,26.02.2013,1,0,30,265.0,3.0


In [6]:
test[(test.shop_id == 12) & (test.item_id == 11373)]

,ID,shop_id,item_id
37296,37296,12,11373


In [7]:

# 将所有出现过的 shop 和 item 做一个结合, 这样就不会漏下任何的组合
grid = []

for block_num in sales["date_block_num"].unique():
    cur_shops = sales.loc[sales["date_block_num"] == block_num, "shop_id"].unique()
    cur_items = sales.loc[sales["date_block_num"] == block_num, "item_id"].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])), dtype="int32"))
index_cols = ["shop_id", "item_id", "date_block_num"]  
grid = pd.DataFrame(np.vstack(grid), columns=index_cols, dtype=np.int32)
grid

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0
5,59,2565,0
6,59,2572,0
7,59,2573,0
8,59,2574,0
9,59,2593,0


# Add the test to the data transformation 

In [8]:
test["date_block_num"] = 34
test

,ID,shop_id,item_id,date_block_num
0,0,5,5037,34
1,1,5,5320,34
2,2,5,5233,34
3,3,5,5232,34
4,4,5,5268,34
5,5,5,5039,34
6,6,5,5041,34
7,7,5,5046,34
8,8,5,5319,34
9,9,5,5003,34


In [9]:
test = test.drop("ID", axis=1)

In [10]:
grid = pd.concat([grid, test],ignore_index=True)
grid


,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0
5,59,2565,0
6,59,2572,0
7,59,2573,0
8,59,2574,0
9,59,2593,0


In [11]:
test_index = grid[grid.date_block_num == 34].index
test_index

Int64Index([10913850, 10913851, 10913852, 10913853, 10913854, 10913855,
            10913856, 10913857, 10913858, 10913859,
            ...
            11128040, 11128041, 11128042, 11128043, 11128044, 11128045,
            11128046, 11128047, 11128048, 11128049],
           dtype='int64', length=214200)

In [12]:
# group the sum of everyday's sale by shop, item, date_block_num
index_cols = ["shop_id", "item_id", "date_block_num"]
gb = sales.groupby(index_cols, as_index=False).agg("sum")
gb.drop("item_price", axis = 1, inplace=True)
gb_price_mean = sales.groupby(index_cols, as_index = False).agg("mean").item_price
gb.rename(columns={"item_cnt_day" : "target"}, inplace=True)
gb["price_mean"] = gb_price_mean.values
gb.head()

,shop_id,item_id,date_block_num,target,price_mean
0,0,30,1,31.0,265.0
1,0,31,1,11.0,434.0
2,0,32,0,6.0,221.0
3,0,32,1,10.0,221.0
4,0,33,0,3.0,347.0


In [13]:
sales[(sales.shop_id ==0) & (sales.item_id == 31) & (sales.date_block_num == 1)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
173403,15.02.2013,1,0,31,434.0,3.0
173454,26.02.2013,1,0,31,434.0,1.0
173455,25.02.2013,1,0,31,434.0,1.0
173456,24.02.2013,1,0,31,434.0,1.0
173457,22.02.2013,1,0,31,434.0,1.0
173458,17.02.2013,1,0,31,434.0,3.0
173459,16.02.2013,1,0,31,434.0,1.0


In [14]:
# merge gb, grid, item categories together
all_data = pd.merge(grid, gb, how = "left", on = index_cols).fillna(0)
all_data = pd.merge(all_data, items, on = ["item_id"], how = "left")

In [15]:
all_data.head()

,shop_id,item_id,date_block_num,target,price_mean,item_name,item_category_id
0,59,22154,0,1.0,999.0,ЯВЛЕНИЕ 2012 (BD),37
1,59,2552,0,0.0,0.0,DEEP PURPLE The House Of Blue Light LP,58
2,59,2554,0,0.0,0.0,DEEP PURPLE Who Do You Think We Are LP,58
3,59,2555,0,0.0,0.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56
4,59,2564,0,0.0,0.0,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59


In [16]:
gb = sales.groupby(["shop_id", "date_block_num"], as_index=False) \
                            .agg({"item_cnt_day":{"shop_block_target_sum":"sum",
                                                 "shop_block_target_mean":np.mean}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how = "left", on = ["shop_id", "date_block_num"]).fillna(0)

In [17]:
gb = sales.groupby(["item_id", "date_block_num"], as_index=False) \
                            .agg({"item_cnt_day":{"item_block_target_sum":"sum",
                                                  "item_block_target_mean":np.mean}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how = "left", on = ["item_id", "date_block_num"]).fillna(0)

In [18]:
sales = pd.merge(sales, items, on = ["item_id"], how = "left")
gb = sales.groupby(["item_category_id", "date_block_num"], as_index = False) \
                            .agg({'item_cnt_day':{'item_cat_block_target_sum':"sum", 
                                                  "item_cat_block_target_mean":np.mean}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how = "left", on = ["item_category_id", "date_block_num"]).fillna(0)

In [19]:
all_data

,shop_id,item_id,date_block_num,target,price_mean,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_mean,item_block_target_sum,item_cat_block_target_mean,item_cat_block_target_sum
0,59,22154,0,1.0,999.000000,ЯВЛЕНИЕ 2012 (BD),37,2017.0,1.092041,1.000000,18.0,1.053414,6094.0
1,59,2552,0,0.0,0.000000,DEEP PURPLE The House Of Blue Light LP,58,2017.0,1.092041,0.000000,0.0,0.969595,287.0
2,59,2554,0,0.0,0.000000,DEEP PURPLE Who Do You Think We Are LP,58,2017.0,1.092041,1.000000,1.0,0.969595,287.0
3,59,2555,0,0.0,0.000000,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,2017.0,1.092041,1.000000,2.0,1.007519,268.0
4,59,2564,0,0.0,0.000000,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,2017.0,1.092041,1.000000,5.0,0.990113,701.0
5,59,2565,0,0.0,0.000000,DEEP PURPLE Stormbringer (фирм.),56,2017.0,1.092041,1.000000,2.0,1.007519,268.0
6,59,2572,0,0.0,0.000000,DEFTONES Koi No Yokan,55,2017.0,1.092041,1.000000,12.0,1.025878,12527.0
7,59,2573,0,0.0,0.000000,DEL REY LANA Born To Die,55,2017.0,1.092041,1.142857,16.0,1.025878,12527.0
8,59,2574,0,2.0,399.000000,DEL REY LANA Born To Die The Paradise Editio...,55,2017.0,1.092041,1.144231,119.0,1.025878,12527.0
9,59,2593,0,0.0,0.000000,DEPECHE MODE Music For The Masses,55,2017.0,1.092041,1.000000,5.0,1.025878,12527.0


In [20]:
all_data.shape

(11128050, 13)

In [21]:
def downcast(df):
    int_col = []
    float_col = []
    for i in df:
        if df[i].dtype == "float64":
            float_col.append(i)
        elif df[i].dtype == "int64":
            int_col.append(i)
    df[float_col] = df[float_col].astype(np.float32)
    df[int_col] = df[int_col].astype(np.int32)
    return df

In [22]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128050 entries, 0 to 11128049
Data columns (total 13 columns):
shop_id                       int64
item_id                       int64
date_block_num                int64
target                        float64
price_mean                    float64
item_name                     object
item_category_id              int64
shop_block_target_sum         float64
shop_block_target_mean        float64
item_block_target_mean        float64
item_block_target_sum         float64
item_cat_block_target_mean    float64
item_cat_block_target_sum     float64
dtypes: float64(8), int64(4), object(1)
memory usage: 1.2+ GB


In [23]:
all_data = downcast(all_data)

In [24]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128050 entries, 0 to 11128049
Data columns (total 13 columns):
shop_id                       int32
item_id                       int32
date_block_num                int32
target                        float32
price_mean                    float32
item_name                     object
item_category_id              int32
shop_block_target_sum         float32
shop_block_target_mean        float32
item_block_target_mean        float32
item_block_target_sum         float32
item_cat_block_target_mean    float32
item_cat_block_target_sum     float32
dtypes: float32(8), int32(4), object(1)
memory usage: 679.2+ MB


In [25]:
del grid, gb

In [26]:
all_data.tail(1)

,shop_id,item_id,date_block_num,target,price_mean,item_name,item_category_id,shop_block_target_sum,shop_block_target_mean,item_block_target_mean,item_block_target_sum,item_cat_block_target_mean,item_cat_block_target_sum
11128049,45,969,34,0.0,0.0,3 ДНЯ НА УБИЙСТВО (BD),37,0.0,0.0,0.0,0.0,0.0,0.0


# Get lag features 

In [27]:
index_cols = ["shop_id", "item_id", "date_block_num", "item_category_id"]
cols_to_rename = list(all_data.columns.difference(index_cols))
cols_to_rename

['item_block_target_mean',
 'item_block_target_sum',
 'item_cat_block_target_mean',
 'item_cat_block_target_sum',
 'item_name',
 'price_mean',
 'shop_block_target_mean',
 'shop_block_target_sum',
 'target']

In [28]:
cols_to_rename.remove("item_name")
cols_to_rename

['item_block_target_mean',
 'item_block_target_sum',
 'item_cat_block_target_mean',
 'item_cat_block_target_sum',
 'price_mean',
 'shop_block_target_mean',
 'shop_block_target_sum',
 'target']

In [29]:
cols_gb_item = [i for i in cols_to_rename if "item_block" in i]
cols_gb_shop = [i for i in cols_to_rename if "shop_block" in i]
cols_gb_cat = [i for i in cols_to_rename if "item_cat" in i]
cols_gb_all = ["target"]
cols_gb_price = ["price_mean"]
cols_gb_key = [["item_id"], ["shop_id"], ["item_category_id"], ["shop_id", "item_id"], ["shop_id", "item_id"]]
cols_gb_value = [cols_gb_item, cols_gb_shop, cols_gb_cat, cols_gb_all, cols_gb_price]
print(cols_gb_value)

[['item_block_target_mean', 'item_block_target_sum'], ['shop_block_target_mean', 'shop_block_target_sum'], ['item_cat_block_target_mean', 'item_cat_block_target_sum'], ['target'], ['price_mean']]


In [30]:
# extract the block, pairs in the cols_gb_value, modify the block number, and merge them according to the block number
# shift

shift_range = [1,2,3,4,6,12]
for month_shift in shift_range:
    for k,v in zip(cols_gb_key,cols_gb_value): 
        index_col = ['date_block_num'] + k
        train_shift = all_data[index_col + v].copy().drop_duplicates()

        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift

        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in v else x
        train_shift = train_shift.rename(columns=foo)
        all_data = pd.merge(all_data, train_shift, on=index_col, how='left').fillna(0)

In [31]:
all_data.columns

Index(['shop_id', 'item_id', 'date_block_num', 'target', 'price_mean',
       'item_name', 'item_category_id', 'shop_block_target_sum',
       'shop_block_target_mean', 'item_block_target_mean',
       'item_block_target_sum', 'item_cat_block_target_mean',
       'item_cat_block_target_sum', 'item_block_target_mean_lag_1',
       'item_block_target_sum_lag_1', 'shop_block_target_mean_lag_1',
       'shop_block_target_sum_lag_1', 'item_cat_block_target_mean_lag_1',
       'item_cat_block_target_sum_lag_1', 'target_lag_1', 'price_mean_lag_1',
       'item_block_target_mean_lag_2', 'item_block_target_sum_lag_2',
       'shop_block_target_mean_lag_2', 'shop_block_target_sum_lag_2',
       'item_cat_block_target_mean_lag_2', 'item_cat_block_target_sum_lag_2',
       'target_lag_2', 'price_mean_lag_2', 'item_block_target_mean_lag_3',
       'item_block_target_sum_lag_3', 'shop_block_target_mean_lag_3',
       'shop_block_target_sum_lag_3', 'item_cat_block_target_mean_lag_3',
       'item_cat

In [32]:
# delete the those rows without enough lags fetures
all_data = all_data[all_data["date_block_num"] >= 12]

In [33]:
all_data.shape

(6639294, 61)

In [34]:
all_data.to_pickle("train_with_test.pickle")

# Special-day features 

In [35]:
all_data["Newyear"] = all_data.date_block_num.apply(lambda x: 1 if x in [12, 24] else 0)
all_data["off_season"] = all_data.date_block_num.apply(lambda x: 1 if x in [16,17,18,19,20,27,28,29,30,31,32] else 0)

In [36]:
all_data = all_data.drop(["item_name"], axis = 1)

In [37]:
all_data.head(1).T

,4488756
shop_id,54.000000
item_id,10297.000000
date_block_num,12.000000
target,4.000000
price_mean,749.000000
item_category_id,37.000000
shop_block_target_sum,8198.000000
shop_block_target_mean,1.295717
item_block_target_mean,1.000000
item_block_target_sum,23.000000


In [38]:
test_lags = all_data.loc[test_index]

In [39]:
test_lags.head().T

,10913850,10913851,10913852,10913853,10913854
shop_id,5.000000,5.000000,5.000000,5.000000,5.000000
item_id,5037.000000,5320.000000,5233.000000,5232.000000,5268.000000
date_block_num,34.000000,34.000000,34.000000,34.000000,34.000000
target,0.000000,0.000000,0.000000,0.000000,0.000000
price_mean,0.000000,0.000000,0.000000,0.000000,0.000000
item_category_id,19.000000,55.000000,19.000000,23.000000,20.000000
shop_block_target_sum,0.000000,0.000000,0.000000,0.000000,0.000000
shop_block_target_mean,0.000000,0.000000,0.000000,0.000000,0.000000
item_block_target_mean,0.000000,0.000000,0.000000,0.000000,0.000000
item_block_target_sum,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
test_lags.drop(["date_block_num", "target",'shop_block_target_mean', 'shop_block_target_sum',
       'item_block_target_sum', 'item_block_target_mean',
       'item_cat_block_target_sum', 'item_cat_block_target_mean'],axis = 1, inplace = True)

In [41]:
test_lags.head().T

,10913850,10913851,10913852,10913853,10913854
shop_id,5.000000,5.000000,5.000000,5.000000,5.000000
item_id,5037.000000,5320.000000,5233.000000,5232.000000,5268.000000
price_mean,0.000000,0.000000,0.000000,0.000000,0.000000
item_category_id,19.000000,55.000000,19.000000,23.000000,20.000000
item_block_target_mean_lag_1,1.000000,0.000000,1.024390,0.933333,0.000000
item_block_target_sum_lag_1,25.000000,0.000000,42.000000,28.000000,0.000000
shop_block_target_mean_lag_1,1.090155,1.090155,1.090155,1.090155,1.090155
shop_block_target_sum_lag_1,1052.000000,1052.000000,1052.000000,1052.000000,1052.000000
item_cat_block_target_mean_lag_1,1.041791,1.022715,1.041791,1.055582,1.625431
item_cat_block_target_sum_lag_1,2443.000000,5808.000000,2443.000000,2203.000000,6596.000000


In [50]:
test_lags.drop("price_mean", axis = 1, inplace = True)
test_lags.reset_index(drop=True, inplace=True)
test_lags.to_pickle("To_update/test.pickle")

In [43]:
test_lags.shape

(214200, 54)

In [44]:
all_data.drop(['shop_block_target_mean', 'shop_block_target_sum',
       'item_block_target_sum', 'item_block_target_mean',
       'item_cat_block_target_sum', 'item_cat_block_target_mean'], axis = 1, inplace=True)

In [45]:
train_lags = all_data.drop(test_index)

In [46]:
train_lags = all_data.drop(["target"], axis = 1)

In [47]:
train_lags.shape

(6639294, 55)

In [52]:
train_lags.drop("price_mean", axis = 1, inplace=True)


NameError: name 'train' is not defined

In [54]:
train_lags.reset_index(drop=True, inplace=True)


In [55]:
train_lags.head().T

,0,1,2,3,4
shop_id,54.000000,54.000000,54.000000,54.000000,54.000000
item_id,10297.000000,10296.000000,10298.000000,10300.000000,10284.000000
date_block_num,12.000000,12.000000,12.000000,12.000000,12.000000
item_category_id,37.000000,38.000000,40.000000,37.000000,57.000000
item_block_target_mean_lag_1,1.000000,1.000000,1.308511,1.058824,1.000000
item_block_target_sum_lag_1,42.000000,24.000000,369.000000,54.000000,4.000000
shop_block_target_mean_lag_1,1.363205,1.363205,1.363205,1.363205,1.363205
shop_block_target_sum_lag_1,10055.000000,10055.000000,10055.000000,10055.000000,10055.000000
item_cat_block_target_mean_lag_1,1.049310,1.018868,1.163135,1.049310,1.016405
item_cat_block_target_sum_lag_1,9959.000000,2052.000000,28598.000000,9959.000000,1425.000000


In [56]:
train_lags.to_pickle("To_update/train_x.pickle")

In [59]:
train_y.reset_index(drop = True, inplace = True)

In [60]:
train_y

0           4.0
1           3.0
2          14.0
3           3.0
4           1.0
5           9.0
6           2.0
7           1.0
8           1.0
9           1.0
10          1.0
11          1.0
12          1.0
13          1.0
14          1.0
15          1.0
16         83.0
17          3.0
18          2.0
19          3.0
20          4.0
21          1.0
22          1.0
23          1.0
24          2.0
25          1.0
26          2.0
27          1.0
28          1.0
29          1.0
           ... 
6639264     0.0
6639265     0.0
6639266     0.0
6639267     0.0
6639268     0.0
6639269     0.0
6639270     0.0
6639271     0.0
6639272     0.0
6639273     0.0
6639274     0.0
6639275     0.0
6639276     0.0
6639277     0.0
6639278     0.0
6639279     0.0
6639280     0.0
6639281     0.0
6639282     0.0
6639283     0.0
6639284     0.0
6639285     0.0
6639286     0.0
6639287     0.0
6639288     0.0
6639289     0.0
6639290     0.0
6639291     0.0
6639292     0.0
6639293     0.0
Name: target, Length: 66

In [61]:
train_y = all_data.target
train_y.to_pickle("To_update/train_y.pickle")